# Byte-Pair Encoding Tokenizer
---
Following Andrej Karpathy's [video](https://www.youtube.com/watch?v=zduSFxRajkE) for tokenizers, building my own BPE tokenizer. From this, will use it as a basis to train my own embedding model based on Word2Vec and analyze the results of that training

## 1) Import Dependencies and Test Data

In [1]:
# Install dependencies
import regex as re
import unittest

# Also defining our regex up here (from GPT-4 tokenizer)
GPT4_SPLIT_PATTERN = r"""'(?i:[sdmt]|ll|ve|re)|[^\r\n\p{L}\p{N}]?+\p{L}+|\p{N}{1,3}| ?[^\s\p{L}\p{N}]++[\r\n]*|\s*[\r\n]|\s+(?!\S)|\s+"""

In [2]:
# First, need to import data. We'll use tiny shakespeare -- will be able to clearly see what kinds of words are commonly used in the ye olde times
file_path = '../data/tinyshakespeare.txt'
with open(file_path, 'r') as file:
    data = file.read()

# Print first 100 chars
print(data[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [3]:
# Let's also define unittests up here so we can call below
class BPETokenizerTest(unittest.TestCase):

    def setUp(self):
        # Setup your regex and BPETokenizer instance here
        self.regex = GPT4_SPLIT_PATTERN
        self.tokenizer = BPETokenizer(self.regex)
        self.text = data[:100000]


    def test_encode(self):
        # Train the tokenizer with some initial text to set up encoding_map
        self.tokenizer.train(self.text, vocab_size=170)
        encoded = self.tokenizer.encode(self.text)
        # Make sure to assert that encoded is not empty and meets expected structure
        self.assertNotEqual(len(encoded), 0, "Encoding should not be empty.")

    def test_decode(self):
        self.tokenizer.train(self.text, vocab_size=200)
        encoded = self.tokenizer.encode(self.text)
        decoded = self.tokenizer.decode(encoded)
        self.assertEqual(decoded, self.text, "Decoded text should match the original.")

    def test_mergepair_effectiveness(self):
        # This will indirectly test _mergepair via the train method
        text = "test test test test"
        self.tokenizer.train(text, vocab_size=130)
        # After training, the tokenizer should have created a token for the repeated sequence
        # The exact token ID might vary, so we check if the length of encoding_map has increased
        self.assertGreater(len(self.tokenizer.encoding_map), 0, "Encoding map should have merged pairs.")

def run_tests():
    suite = unittest.TestLoader().loadTestsFromTestCase(BPETokenizerTest)
    unittest.TextTestRunner(verbosity=2).run(suite)

In [126]:
# Defining our TrieNode and associated functions to help us speed up encoding
class TrieNode:
    def __init__(self):
        self.children = {}
        self.token = None

def add_to_trie(root, ascii_sequence, token):
    current = root
    for ascii_val in ascii_sequence:
        if ascii_val not in current.children:
            current.children[ascii_val] = TrieNode()
        current = current.children[ascii_val]
    current.token = token

def find_longest_matching_token(root, ascii_sequence):
    current = root
    last_token = None
    for ascii_val in ascii_sequence:
        if ascii_val in current.children:
            current = current.children[ascii_val]
            if current.token:
                last_token = current.token
        else:
            break
    return last_token

## 2) Defining our BPETokenizer Class

### Approach
---
For our tokenizer, we'll create a class called BPETokenizer. Upon instantiation, it will take in a regex expression (optional). Subfunctions include:
1. Various getters and setters
2. _encodetext(text): Takes in raw text and returns 2-D python list of ASCII values (if regex, split by regex; else just 2-D w/ 1 element)
3. _mergepair(encodedtext, token_pair): helper function to take in your encoded text and a tuple (pair, token) that replaces the pair with the desired token
4. train(text, vocab_size): Function to iteratively convert most frequent byte-pairs to a new token. Saves encoding map to class
5. _optimize_mappings: Function that creates our TrieTree (for fast encoding) and an optimized decoding mapping (for fast decoding)
6. encode(text): Converts text to tokens; returns as 1-d python list of tokens
7. decode(tokens): Converts tokens to text; returns string.

In [129]:
class BPETokenizer():

    def __init__(self, regex=None):
        self.encoding_map = {}
        self.decoding_map = {}
        self.regex = regex

    def regex_setter(self, re):
        """ Simple setter function for regex """
        self.regex = re

    def encmap_getter(self):
        """ Simple getter to return our encoding map """
        return self.encoding_map
        
    def decmap_getter(self):
        """ Simple getter to return our decoding map """
        return self.decoding_map

    def _encodetext(self, text):
        """
            Takes in raw text and returns 2-D python array of ASCII values (e.g., [[70, 105, 114, 115, 116], [32, 67,...]])
            Replaces non-ASCII with '?'
        """
        if self.regex == None:
            text = [text]
        else:
            text = re.findall(self.regex, text)     # Converts text to ['First', ' Citizen', ':\n', 'Before', ' we', ' proceed',...]
        return list( list(t.encode("ascii", errors="replace")) for t in text )

    def _mergepair(self, tokens, pair_tok):
        """
            Takes in 2-D python list of lists of tokens and iterates through, replacing 'tok' where 'pair' exists
        """
        pair, tok = pair_tok
        merged_tokens = []
        for block in tokens:
            merged_block = []
            skip_next_idx = False
            if (len(block) < 2):
                merged_block = block           # Simply keep our block the same if not 2+ elements
            else:
                for idx in range(len(block) - 1):
                    if skip_next_idx:
                        skip_next_idx = False
                        if idx == len(block) - 2:
                            merged_block.append(block[idx + 1])
                    elif block[idx] == pair[0] and block[idx + 1] == pair[1]:
                        merged_block.append(tok)  # Merge pair into 'tok'
                        skip_next_idx = True  # Skip the next index as it's already merged
                        if idx == len(block) - 2:  # Check if at the end after merging
                            break  # No need to append anything else, as the pair was the last two elements
                    else:
                        merged_block.append(block[idx])
                        if idx == len(block) - 2:  # Handle the last element if it's not part of a pair
                            merged_block.append(block[idx + 1])

            merged_tokens.append(merged_block)
        return merged_tokens

    def train(self, text, vocab_size):
        """
            Takes in raw text and a desired length for max vocabulary size
            Upon completion, sets encoder_map and decoder_map to the BPE mappings (forward, backward resp.) 
        """
        encoded_text = self._encodetext(text)
        encoding_map = {}                 # Create python dictionary of merges
        num_merges = vocab_size - 128     # Number of iterations of BPE
        
        for i in range(num_merges):
            bytepair_count = {}
            for block in encoded_text:
                if len(block) > 1:
                    for idx in range(len(block)-1):
                        pair = (block[idx], block[idx+1])
                        count = bytepair_count.get(pair, 0)
                        bytepair_count[pair] = count+1
            
            # Once done iterating through all the ascii values, sort and assign most freq bytepair to new token
            freq_pair = max(bytepair_count, key=bytepair_count.get)
            new_token = 128 + i
            encoding_map[freq_pair] = new_token
            encoded_text = self._mergepair(encoded_text, (freq_pair, new_token))

        self.encoding_map = encoding_map
        self.decoding_map = {value: key for key, value in encoding_map.items()}
        self._optimize_mapping()   # Run our mapping optimizer so we can more efficiently encode / decode in the future

    def _optimize_mapping(self):
        """
            Creates self.optimized_encoding_map and self.optimized_decoding_map from their primitive parent mappings.
            These optimized mappings allow us to compute these optimized mappings once and save time on calling encode / decode:
                encode(): finds longest token that matches the initial encodings and replaces; prevents iterative process
                decode(): goes directly from token to full length of chars (rather than taking iterative steps)
        """
        optimized_decoding_map = {i: [i] for i in range(128)}
        decoding_map_list = list(self.decoding_map.items())
        for tok, pair in decoding_map_list:
            expanded_pair = [pair[0], pair[1]]
            while max(expanded_pair)>127:
                for i, pair_token in enumerate(expanded_pair):
                    expanded_pair[i:i+1] = optimized_decoding_map[pair_token]
            optimized_decoding_map[tok] = expanded_pair
        
        self.optimized_decoding_map = optimized_decoding_map
        self.optimized_encoding_map = {tuple(value): key for key, value in optimized_decoding_map.items()}
        # Defining our TrieNode as well to speed up encoding
        self.trie_root = TrieNode()
        for ascii_sequence, token in self.optimized_encoding_map.items():
            add_to_trie(self.trie_root, ascii_sequence, token)

    def encode(self, text):
        """
            Takes in raw text and returns tokenized text (1-D array)
        """
        encoded_text = self._encodetext(text)
        tokenized_text = []
        for block in encoded_text:
            next_idx = 0
            while next_idx < len(block):
                token = find_longest_matching_token(self.trie_root, block[next_idx:])
                next_idx = next_idx + len(self.optimized_decoding_map[token])
                tokenized_text.append(token)
        return tokenized_text

    def decode(self, tokens):
        """
            Takes in tokenized text (1-D array) and returns raw text
        """
        token_text = []
        for i in tokens:
            token_text.extend(self.optimized_decoding_map[i])
        decoded_text = ''.join(chr(value) for value in token_text)
        return decoded_text

In [130]:
# Running our tests 
run_tests()

test_decode (__main__.BPETokenizerTest.test_decode) ... ok
test_encode (__main__.BPETokenizerTest.test_encode) ... ok
test_mergepair_effectiveness (__main__.BPETokenizerTest.test_mergepair_effectiveness) ... ok

----------------------------------------------------------------------
Ran 3 tests in 3.481s

OK


In [131]:
# Define our data we'll be trianing on and tokenize it
text_data = data[:2000]
tokenizer = BPETokenizer(GPT4_SPLIT_PATTERN)
tokenizer.train(text_data, 200)
tokenized_text = tokenizer.encode(text_data)

In [132]:
print(tokenizer.decode(tokenized_text[:200]))

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talkin


## 3) Analyze our Results

In [133]:
def print_tokens(encoding_map):
    """
    Takes in a dictionary called encoding_map and prints out the text that each token relates to
    """
    encodings = list(encoding_map.items())
    tok_text = {}
    for (tok_1, tok_2), mapped_token in encodings:
        text = []
        # Append the text representation of tok_1
        if tok_1 in tok_text:
            text.append(tok_text[tok_1])
        else:
            text.append(chr(tok_1))
        # Append the text representation of tok_2
        if tok_2 in tok_text:
            text.append(tok_text[tok_2])
        else:
            text.append(chr(tok_2))
        # Join the characters or strings in 'text' list into a single string
        tok_text[mapped_token] = ''.join(text)
    
    tok_text_list = list(tok_text.items())
    for tok, text in tok_text_list:
        # Replace newline characters with a visible representation
        safe_text = text.replace('\n', '\\n')
        print(f"Token: {tok}, Text: '{safe_text}'")

In [134]:
# Let's print out the text associated with our tokens that we encoded
# print_tokens(tokenizer.encmap_getter())

Token: 128, Text: ' t'
Token: 129, Text: 'he'
Token: 130, Text: 'it'
Token: 131, Text: 'en'
Token: 132, Text: 'ou'
Token: 133, Text: ' a'
Token: 134, Text: ' w'
Token: 135, Text: 're'
Token: 136, Text: ':\n'
Token: 137, Text: '\n\n'
Token: 138, Text: 'st'
Token: 139, Text: ' C'
Token: 140, Text: 'is'
Token: 141, Text: 'on'
Token: 142, Text: 'iti'
Token: 143, Text: ' the'
Token: 144, Text: ' h'
Token: 145, Text: 'itiz'
Token: 146, Text: 'itizen'
Token: 147, Text: 'at'
Token: 148, Text: 'ir'
Token: 149, Text: ' to'
Token: 150, Text: ' c'
Token: 151, Text: 'in'
Token: 152, Text: ' Citizen'
Token: 153, Text: ' p'
Token: 154, Text: ' s'
Token: 155, Text: '.\n\n'
Token: 156, Text: 'irst'
Token: 157, Text: ' he'
Token: 158, Text: 'll'
Token: 159, Text: 'us'
Token: 160, Text: 'or'
Token: 161, Text: ' b'
Token: 162, Text: ' f'
Token: 163, Text: 'First'
Token: 164, Text: 've'
Token: 165, Text: 'no'
Token: 166, Text: ' we'
Token: 167, Text: ' m'
Token: 168, Text: ' th'
Token: 169, Text: 'ic'
Toke

In [140]:
# We can check the compression rate
print(f"Length of original string:     {len(text_data)}")
print(f"Length of tokenized string:    {len(tokenized_text)}")
print(f"Compression (% of original):   {((len(tokenized_text) / len(text_data))*100):.2f}%")
print("\n")

# Let's make sure the text is properly decodable as well
# print(tokenizer.decode(tokenized_text[:200]))

Length of original string:     2000
Length of tokenized string:    1132
Compression (% of original):   56.60%




## 4) Optimizing our Code and Analyzing Runtime Performance
---
Able to successfully optimize the code to run significantly (>100x) faster on encoding runs. Detail below.

### Define Dependencies and Helper Functions

In [15]:
# Import libraries to analyze runtime performance 
import cProfile
import pstats
import io
import time as pytime
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from matplotlib.patches import Patch
import pickle
from datetime import datetime
import os

In [16]:
def plot_time_vs_configurations(run_times, metric_label="Metric", varying_param_label="Parameter"):
    """
    Plots times versus configurations for a given metric, with configurations represented as tuples.
    
    Parameters:
        run_times (dict): A dictionary where keys are tuples representing configurations (e.g., (metric_size, varying_param))
                          and values are times.
        metric_label (str): The label for the x-axis, indicating the metric being measured (default is 'Metric').
        varying_param_label (str): The description for varying parameters, used in legend labeling (default is 'Parameter').
    """
    plt.figure(figsize=(5, 3))
    
    # Extract metric sizes and varying parameters from dictionary keys
    metric_sizes = sorted(set(key[0] for key in run_times.keys()))
    varying_params = sorted(set(key[1] for key in run_times.keys()))
    
    for param in varying_params:
        times = [run_times.get((metric, param), None) for metric in metric_sizes]
        plt.plot(metric_sizes, times, marker='o', label=f"{varying_param_label} {param}")
    
    plt.title(f"Time vs. {metric_label}")
    plt.xlabel(metric_label)
    plt.ylabel("Time (seconds)")
    plt.legend()
    plt.grid(True)
    plt.show()

In [17]:
def train_polynomial_model_from_dict(data_dict, degree=2):
    """
    Trains a polynomial regression model from a dictionary where keys are tuples representing features
    and values are targets.
    
    Parameters:
        data_dict (dict): A dictionary with keys as tuples representing feature values and values as target values.
        degree (int): The degree of the polynomial features.
        
    Returns:
        (X, y): Tuple of our data used in our training
        poly_model (LinearRegression): The trained polynomial regression model.
        poly (PolynomialFeatures): The polynomial feature transformer used for the model.
    """
    # Extract features (X) and targets (y) from the dictionary
    X = np.array(list(data_dict.keys()))
    y = np.array(list(data_dict.values()))

    # Transform your features into polynomial features
    poly = PolynomialFeatures(degree=degree)
    X_poly = poly.fit_transform(X)

    # Fit the linear regression model on polynomial features
    poly_model = LinearRegression()
    poly_model.fit(X_poly, y)
    
    return (X, y), poly_model, poly

In [18]:
def performance_over_optimizations_observed(results_list):
    """
    Prints the average performance improvement across all configurations for each run,
    compared to the initial run.
    """
    initial_time = sum(results_list[0].values())
    for i, run_data in enumerate(results_list[1:], 1):
        current_time = sum(run_data.values())
        percentage = (current_time / initial_time) * 100
        improvement_factor = 100 / percentage
        print(f"Optimization {i}: {percentage:.2f}% of initial time; ({improvement_factor:.2f}x speed improvement).")

def performance_over_optimizations_predictions(number_list):
    """Prints the improvement factor over the initial value for a series of optimization runs."""
    if not number_list:
        print("The list is empty.")
        return

    initial = number_list[0]
    for i, value in enumerate(number_list[1:], 1):
        percentage = (value / initial) * 100
        improvement_factor = initial / value
        print(f"Optimization {i}: {percentage:.2f}% of initial time; ({improvement_factor:.2f}x speed improvement).")


In [19]:
def plot_polynomial_regression_surface(X, y, poly, poly_model, feature_labels=['Feature 1', 'Feature 2'], target_label='Target'):
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')

    # Scatter plot of original data points
    scatter = ax.scatter(X[:, 0], X[:, 1], y, color='blue', label='Actual Data')

    # Generate mesh for the surface
    x1_range = np.linspace(X[:, 0].min(), X[:, 0].max(), 20)
    x2_range = np.linspace(X[:, 1].min(), X[:, 1].max(), 20)
    x1_mesh, x2_mesh = np.meshgrid(x1_range, x2_range)
    mesh_flat = np.c_[x1_mesh.ravel(), x2_mesh.ravel()]
    mesh_poly = poly.transform(mesh_flat)  # Transform to polynomial features
    y_mesh = poly_model.predict(mesh_poly)  # Predict using the polynomial model
    y_mesh = y_mesh.reshape(x1_mesh.shape)

    # Plot the surface
    surface = ax.plot_surface(x1_mesh, x2_mesh, y_mesh, color='orange', alpha=0.5, edgecolor='none')

    ax.set_xlabel(feature_labels[0])
    ax.set_ylabel(feature_labels[1])
    ax.set_zlabel(target_label)
    ax.set_title(f'3D Plot of {feature_labels[0]}, {feature_labels[1]} vs. {target_label} with Polynomial Regression Surface')

    # Manually define legend
    scatter_legend = Patch(color='blue', label='Actual Data')
    surface_legend = Patch(color='orange', label='Polynomial Regression Model')
    plt.legend(handles=[scatter_legend, surface_legend])

    plt.show()

In [20]:
def training_run(datasizes, vocabsizes, printprog = True):
    """ Function that completes a training run for multiple datasizes and vocabsizes(lists of params). Returns the data for each run as a dict {(tuple of params (D, V)): Time} """
    dict_results = {}
    for d in datasizes:
        runtime_testdata = data[:d]
        for v in vocabsizes:
            rt_tokenizer = BPETokenizer(GPT4_SPLIT_PATTERN)
            
            start_time = pytime.time()  # Start timer
            rt_tokenizer.train(runtime_testdata, v)
            end_time = pytime.time()  # End timer
            
            run_time = end_time - start_time
            dict_results[(d, v)] = run_time  # Save the time with (d, v) as the key
            if printprog:
                print(f"D: {d}, V: {v}; Time: {run_time:.2f}s")    
    
    return dict_results

def encoding_run(vocabsizes, encodinglengths, traindata, printprog = True):
    """ Function that completes an encoding run for multiple vocabsizes and encodinglengths (lists of params). Returns the data for each run as a dict {(tuple of params (V, L)): Time} """
    dict_results = {}
    for v in vocabsizes:
        rt_tokenizer = BPETokenizer(GPT4_SPLIT_PATTERN)
        rt_tokenizer.train(traindata, v)
        
        for l in encodinglengths:
            start_time = pytime.time()  # Start timer
            rt_tokenizer.encode(data[:l])
            end_time = pytime.time()  # End timer
            run_time = end_time - start_time
            dict_results[(v, l)] = run_time  # Save the time with (l, v) as the key
            print(f"V: {v}, L: {l}; Time: {run_time:.2f}s")

    return dict_results

def pred_data(datapoint, polyfunc, polymodel):
    """ Function that prints and returns the predicted time for the input data based on the trained polynomial fit """
    datapoint_poly = polyfunc.transform(datapoint)
    prediction = polymodel.predict(datapoint_poly)[0]
    print(f"Predicted Time: {prediction:.2f} seconds")
    return prediction

### Progress in Optimization
---
Keeping a log of key changes made during each step:
1. Initial run using our working implementation (un-optimized)
2. Updated the mergepair function to be a bit more efficient (small edits)
3. Implemented the Trie Node encoding function to speed up encoding. Also implemented fast decoding (compute token - chars for all tokens so just one pass through lets you decode).

*Results below, but note that #3 actually increases training time because we now have an '_optimize_mapping' function that sets up our TrieNode / optimized decoding map.*


**Improvement for Training Runs**  
\------------------------------------------------  

Performance Over Observed Data (Avg)  
\================================================  
Optimization 1: 91.80% of initial time; (1.09x speed improvement).  
Optimization 2: 107.38% of initial time; (0.93x speed improvement).  

Performance Over Predicted Time for Large Run  
\================================================  
Optimization 1: 89.64% of initial time; (1.12x speed improvement).  
Optimization 2: 103.56% of initial time; (0.97x speed improvement).  


**Improvement for Encoding Runs**  
\------------------------------------------------  

Performance Over Observed Data (Avg)  
\================================================  
Optimization 1: 89.11% of initial time; (1.12x speed improvement).  
Optimization 2: 0.97% of initial time; (102.62x speed improvement).  

Performance Over Predicted Time for Large Run  
\================================================  
Optimization 1: 91.48% of initial time; (1.09x speed improvement).  
Optimization 2: 0.44% of initial time; (226.54x speed improvement).  

In [21]:
# Let's first define our existing performance
# On our training run
trainingrun_testdata = np.array([[len(data), 2048]])  # (Data_size, vocab_size)
trainingrun_predlist     = []
trainingrun_resultslist  = []

# On our encoding runs
encodingrun_testdata = np.array([[2048, 1000000]])  # (Vocab_size, Encoding_length)
encodingrun_predlist     = []
encodingrun_resultslist  = []

In [136]:
# After updating our code, we can run this to append our performance to our prior list
train_data_sizes      = [10000, 30000, 100000, 300000]
train_vocab_sizes     = [200, 300, 500, 1000]

trainingrun_results = training_run(train_data_sizes, train_vocab_sizes, printprog = True)
trainingrun_resultslist.append(trainingrun_results)
train_data, train_polymodel, train_poly = train_polynomial_model_from_dict(trainingrun_results, degree=2)
train_pred = pred_data(trainingrun_testdata, train_poly, train_polymodel)
trainingrun_predlist.append(train_pred)

# And for our encoding run
encoding_traindata   = data[:10000]
encoding_vocab_sizes = [200, 300, 500, 1000]
encoding_lengths     = [10000, 30000, 100000, 300000]

encodingrun_results = encoding_run(encoding_vocab_sizes, encoding_lengths, encoding_traindata, printprog = True)
encodingrun_resultslist.append(encodingrun_results)
encoding_data, encoding_polymodel, encoding_poly = train_polynomial_model_from_dict(encodingrun_results, degree=2)
encoding_pred = pred_data(encodingrun_testdata, encoding_poly, encoding_polymodel)
encodingrun_predlist.append(encoding_pred)

D: 10000, V: 200; Time: 0.17s
D: 10000, V: 300; Time: 0.29s
D: 10000, V: 500; Time: 0.49s
D: 10000, V: 1000; Time: 0.82s
D: 30000, V: 200; Time: 0.54s
D: 30000, V: 300; Time: 1.21s
D: 30000, V: 500; Time: 1.97s
D: 30000, V: 1000; Time: 3.54s
D: 100000, V: 200; Time: 1.94s
D: 100000, V: 300; Time: 4.28s
D: 100000, V: 500; Time: 7.65s
D: 100000, V: 1000; Time: 14.75s
D: 300000, V: 200; Time: 6.80s
D: 300000, V: 300; Time: 13.78s
D: 300000, V: 500; Time: 25.13s
D: 300000, V: 1000; Time: 46.74s
Predicted Time: 373.52 seconds
V: 200, L: 10000; Time: 0.00s
V: 200, L: 30000; Time: 0.02s
V: 200, L: 100000; Time: 0.06s
V: 200, L: 300000; Time: 0.15s
V: 300, L: 10000; Time: 0.00s
V: 300, L: 30000; Time: 0.02s
V: 300, L: 100000; Time: 0.03s
V: 300, L: 300000; Time: 0.16s
V: 500, L: 10000; Time: 0.00s
V: 500, L: 30000; Time: 0.02s
V: 500, L: 100000; Time: 0.03s
V: 500, L: 300000; Time: 0.16s
V: 1000, L: 10000; Time: 0.05s
V: 1000, L: 30000; Time: 0.02s
V: 1000, L: 100000; Time: 0.02s
V: 1000, L: 3

In [137]:
# Let's see how our performance has been improving for training runs
print("Improvement for Training Runs\n---------------------------------------------------")
print("\nPerformance Over Observed Data (Avg)\n================================================")
training_run_performance = performance_over_optimizations_observed(trainingrun_resultslist)
print("\nPerformance Over Predicted Time for Large Run\n================================================")
performance_over_optimizations_predictions(trainingrun_predlist)
print("\n\n")

# And for encoding runs
print("Improvement for Encoding Runs\n---------------------------------------------------")
print("\nPerformance Over Observed Data (Avg)\n================================================")
training_run_performance = performance_over_optimizations_observed(encodingrun_resultslist)
print("\nPerformance Over Predicted Time for Large Run\n===================================================")
performance_over_optimizations_predictions(encodingrun_predlist)

Improvement for Training Runs
---------------------------------------------------

Performance Over Observed Data (Avg)
Optimization 1: 91.80% of initial time; (1.09x speed improvement).
Optimization 2: 107.38% of initial time; (0.93x speed improvement).

Performance Over Predicted Time for Large Run
Optimization 1: 89.64% of initial time; (1.12x speed improvement).
Optimization 2: 103.56% of initial time; (0.97x speed improvement).



Improvement for Encoding Runs
---------------------------------------------------

Performance Over Observed Data (Avg)
Optimization 1: 89.11% of initial time; (1.12x speed improvement).
Optimization 2: 0.97% of initial time; (102.62x speed improvement).

Performance Over Predicted Time for Large Run
Optimization 1: 91.48% of initial time; (1.09x speed improvement).
Optimization 2: 0.44% of initial time; (226.54x speed improvement).


In [139]:
# Optional code that will save our data to a folder
def save_optimization_results():
    filename = datetime.now().strftime("%Y-%m-%d_%H-%M-%S") + '.pkl'
    filepath = os.path.join('optimization_results', filename)
    data_to_save = {
        'trainingrun_testdata': trainingrun_testdata,
        'trainingrun_predlist': trainingrun_predlist,
        'trainingrun_resultslist': trainingrun_resultslist,
        'encodingrun_testdata': encodingrun_testdata,
        'encodingrun_predlist': encodingrun_predlist,
        'encodingrun_resultslist': encodingrun_resultslist
    }
 
    with open(filepath, 'wb') as file:
        pickle.dump(data_to_save, file)
    
    print(f"Data saved to {filepath}")

# save_optimization_results()

Data saved to optimization_results\2024-03-06_18-40-22.pkl


### Analyze the Time for Our Training Runs
---
Scales with a slight parabolic trend -- especially as datasize grows beyond 100k chars and vocab size grows beyond 500.  
**Post-Optimization**: There are some optimizations we could make to run this quicker -- notably around parallelization and more efficiently counting the occurances of our pairs -- but because we only call train once and these are not likely to lead to huge speed improvements (except at large scale that is beyond what my laptop is capable of doing), this doesn't seem worth the work.

In [ ]:
# Let's compute the time required to run based on size of dataset and size of vocabulary and see how they grow
# train_run_times = training_run(data_sizes, vocab_sizes, printprog = False)

In [ ]:
# Can plot our Training Time vs. Data Size 
plot_time_vs_configurations(trainingrun_resultslist[-1], metric_label="Data Size", varying_param_label="Time")

In [ ]:
# Train our polynomial fit
train_data, train_polymodel, train_poly = train_polynomial_model_from_dict(trainingrun_resultslist[-1], degree=2)

# Predict based on our data
train_pred = pred_data(trainingrun_testdata, train_poly, train_polymodel)

# Visualize our graph
plot_polynomial_regression_surface(train_data[0], train_data[1], train_poly, train_polymodel, feature_labels=['Data Size', 'Vocab Size'], target_label='Time')

### Analyzing Runtime of Encoding
---
We can see that the runtime of encoding grows with slight parabolic trajectory in our base implementation. We'll work to reduce this above in the optimization section.  
**Post-Optimization:** We are able to reduce this to trivial time by using a TrieNode. For example, encoding 1M chars using a vocab size of 2048 previously would have taken ~4 minutes, now it runs in around 1 second.

In [ ]:
# Let's compute the time required to run based on size of dataset and size of vocabulary and see how they grow
# encodingrun_results = encoding_run(encoding_vocab_sizes, encoding_lengths, printprog = True)

In [ ]:
# Can plot our Training Time vs. Encoding Length 
plot_time_vs_configurations(encodingrun_resultslist[-1], metric_label="Encoding Length", varying_param_label="Time")

In [ ]:
# Train our polynomial fit
encoding_data, encoding_polymodel, encoding_poly = train_polynomial_model_from_dict(encodingrun_resultslist[-1], degree=2)

# Predict based on our data
encoding_pred = pred_data(encodingrun_testdata, encoding_poly, encoding_polymodel)

# Visualize our graph
plot_polynomial_regression_surface(encoding_data[0], encoding_data[1], encoding_poly, encoding_polymodel, feature_labels=['Vocab Size', 'Encoding Length'], target_label='Time')

### Other Runtime Analysis Options

In [ ]:
# Define our code that we want to test runtime on
runtime_testdata = data[:10000]
rt_tokenizer = BPETokenizer(GPT4_SPLIT_PATTERN)

def profile_your_function():
    rt_tokenizer.train(runtime_testdata, 500)
    # rt_tokenized_text = rt_tokenizer.encode(runtime_testdata)
    

In [ ]:
# Create a Profile object
pr = cProfile.Profile()
pr.enable()  # Start collecting profiling data

# Call the function or part of the code you want to profile
profile_your_function()

pr.disable()  # Stop collecting profiling data

# Use io.StringIO to capture the profiling output
s = io.StringIO()
# Sort the statistics by cumulative time spent and print the top parts
ps = pstats.Stats(pr, stream=s).sort_stats('cumulative')
ps.print_stats()

# Print the profiling result
print(s.getvalue())